# Modelo

Neste notebook será desenvolvido o modelo de ações com o foco em prever os dados das ações em um dia futuro a partir das informações de hoje. Será usado o modelo Lasso Regression como apresentado e justificado no arquivo Trial_and_Error - Model Selection

In [1]:
import pandas as pd
import numpy as np
import matplotlib. pyplot as plt
import matplotlib


In [ ]:
from joblib import dump
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import linear_model
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Preparo dos Dados

Após um processo iterativo à base de tentativa e erro, foi descoberta a necessidade de normalizar também os dados a serem previstos. Para isso, os dados nas tabelas foram convertidos para uma gaussiana (partindo do pressuposto que os dados do mercado de ações podem ser convertido neste modelo) e média e desvio padrão foram extraídos da tabela original e salvos em um arquivo à parte. Para o caso das datas, foi definido arbitrariamente que a média usada seria a média aritmética entre o primeiro e o último dia com desvio padrão igual a metade da média. Além disso, o volume foi removido das previsões.

Alternativas pensadas foram dividir as datas (número de maior escala no modelo) em dia, mês e ano e [codificá-los](https://contrib.scikit-learn.org/category_encoders/). Essa estratégia foi descartada por medo que a quantidade de colunas adicionadas afetasse o modelo que já conta com dezenas de milhões de dados. A estratégia de usar mínimo e máximo ao invés de média e desvio padrão também foi considerada, mas opto pela gaussiana por ter experiência prévia com o funcionamento desse tipo de transformação.

In [3]:
df = pd.read_csv("filteredStocks.csv")
df

,Unnamed: 0,FullName,Date,Open,High,Low,Close,Volume
0,0,Apple,20170103,52.4880,52.8270,52.3780,52.6630,84271282
1,33889,Barclays,20170103,9.1999,9.4240,9.1912,9.2931,5515413
2,185738,Chipotle Mexican Grill,20170103,343.7000,350.4900,340.0000,341.2700,728093
3,35365,Brandywine Realty Trust,20170103,7.4194,7.5758,7.3332,7.4036,1538256
4,184262,Devon Energy,20170103,59.6550,61.2540,59.1570,60.8060,5952293
...,...,...,...,...,...,...,...,...
210825,169501,Suncor Energy,20221110,36.2100,36.4800,35.9700,36.4200,3287248
210826,56024,Bank of Montreal,20221110,77.7800,77.7800,76.8500,77.2200,214921
210827,160645,NXP Semiconductors,20221110,115.2600,116.2500,115.0400,116.0500,2392787
210828,85370,Coca-Cola,20221110,46.2000,46.6400,46.1350,46.5400,8022883


In [4]:
df_1=pd.read_csv("filteredStocks_1.csv", na_values=['null'],parse_dates=True,infer_datetime_format=True)
df_1.head()

,Date,FullName,Open,High,Low,Close,Volume
0,20170103,Apple,52.488,52.827,52.378,52.663,84271282
1,20170103,Petrobras,24.131,25.044,24.074,24.826,13414486
2,20170103,Target,43.792,43.807,42.845,43.142,9038799
3,20170103,Southern Company,35.279,35.449,34.487,34.655,9102867
4,20170103,Intuitive Surgical,157.000,157.980,155.720,157.240,1033566


In [5]:
df_2=pd.read_csv("filteredStocks_2.csv", na_values=['null'],parse_dates=True,infer_datetime_format=True)
df_2.head()

,Date,FullName,Open,High,Low,Close,Volume
0,20200102,Abbott Laboratories,42.235,42.422,41.664,41.908,3446924
1,20200102,Oracle,42.983,43.194,42.029,42.371,15766697
2,20200102,Boeing,120.300,120.990,118.460,119.270,4679018
3,20200102,Cadence Design Systems,19.150,19.240,18.570,18.830,3508565
4,20200102,Edward Lifesciences,64.095,64.905,63.305,63.860,1215780


In [6]:
# Listando todas as companhias no dataFrame
name_list = df_1['FullName'].unique()
print(name_list)

['Apple' 'Petrobras' 'Target' 'Southern Company' 'Intuitive Surgical'
 'Dominion Energy' 'Micron Technology' 'Fomento Económico Mexicano'
 'Canadian Pacific Railway' 'HDFC Bank' 'Edward Lifesciences'
 'Goldman Sachs' 'Waste Management' 'Netflix' 'Global Payments'
 'Starbucks' 'Cisco' '3M' 'IBM' 'Vale' 'HSBC' 'NetEase'
 'Royal Bank Of Canada' 'Honeywell' 'Texas Instruments'
 'Toronto Dominion Bank' 'Nextera Energy' 'Intel' 'QUALCOMM' 'Fedex'
 'Nike' 'Fidelity National Information Services'
 'Canadian Natural Resources' 'Occidental Petroleum' 'Unilever'
 'Johnson & Johnson' 'Ecolab' 'Ford' 'Astrazeneca' 'Synopsys'
 'Marathon Petroleum' 'Pepsico' 'National Grid' 'Oracle' 'Pfizer'
 'Mastercard' 'Wallmart' 'Sempra Energy' 'Coca-Cola' 'Ambev'
 'Vertex Pharmaceutical' 'Vmware' 'Visa' 'General Motors' 'Exxon Mobil'
 'Tesla' 'NXP Semiconductors' 'Microsoft' 'Cadence Design Systems'
 'Cheesecake Factory Inc' 'Canon' 'McDonald' 'The Hershey Company'
 'Capital One' 'Citigroup' 'Compania de Minas B

## Conversão dos dados
Aqui pos dados preparados são salvos em dois dataFrames diferentes. Também é extraída a média e o desvio padrão de cada categoria para permitir que qualquer pessoa com acesso a esses dados consiga converter os valores para uma gaussiana antes de fazer a previsão. Então os dados estatísticos são salvos em um arquivo com formato json e os dois dataframes são combinados em um único de forma cruzada.

In [7]:
# Agrupando por empresa para utilizar nos experimentos
d_2 = dict();
d_1 = dict();
stats = dict();
for name in name_list:
    d_1[name] = df_1.loc[df_1["FullName"] == name].drop(columns=["FullName"])
    d_2[name] = df_2.loc[df_2["FullName"] == name].drop(columns=["FullName"])
    d_2[name] = d_2[name].drop(columns = ["Open", "Close", "Volume"])
    d_2[name].rename(columns = {'Date':'Date-1', 'High':'High-1', "Low":"Low-1"}, inplace = True)
    stats[name]=dict();
    stats[name]["Open"]=dict();
    stats[name]["High"]=dict();
    stats[name]["Low"]=dict();
    stats[name]["Close"]=dict();
    
    stats[name]["Open"]["Mean"] = np.mean(df.loc[df["FullName"]==name].Open)
    stats[name]["Open"]["Std"] = np.std(df.loc[df["FullName"]==name].Open)
    stats[name]["High"]["Mean"] = np.mean(df.loc[df["FullName"]==name].High)
    stats[name]["High"]["Std"] = np.std(df.loc[df["FullName"]==name].High)
    stats[name]["Low"]["Mean"] = np.mean(df.loc[df["FullName"]==name].Low)
    stats[name]["Low"]["Std"] = np.std(df.loc[df["FullName"]==name].Low)
    stats[name]["Close"]["Mean"] = np.mean(df.loc[df["FullName"]==name].Close)
    stats[name]["Close"]["Std"] = np.std(df.loc[df["FullName"]==name].Close)
    

d_1[name_list[0]]

,Date,Open,High,Low,Close,Volume
0,20170103,52.488,52.827,52.378,52.663,84271282
247,20170104,52.516,53.105,52.414,52.946,72548556
304,20170105,53.140,53.599,52.849,53.534,75572081
523,20170106,53.757,54.138,53.687,54.094,88773291
645,20170109,54.491,54.778,53.959,54.008,109831110
...,...,...,...,...,...,...
106987,20191224,106.570,106.700,106.040,106.040,15296050
107201,20191226,106.130,108.400,106.040,107.910,35610489
107366,20191229,107.720,108.640,107.630,107.840,29133164
107403,20191230,107.570,107.840,106.140,106.510,31555587


In [8]:
import json

with open('stats.json', 'w') as handle:
    j = json.dumps(stats)
    handle.write(j)
    handle.close()


In [10]:
d_2[name_list[0]]

,Date-1,High-1,Low-1
8,20200102,105.49,101.620
146,20200105,102.83,99.790
354,20200106,101.69,99.044
561,20200107,102.42,100.990
632,20200108,106.17,102.880
...,...,...,...
102558,20221106,174.36,171.100
102639,20221107,174.51,173.290
102818,20221108,175.61,173.710
102986,20221109,175.46,172.520


In [11]:
test_frame = dict()
for name in name_list:
    test_frame[name] = d_1[name].merge(d_2[name], how = "cross")


## Geração e registro de modelos
Na próxima (e última) etapa os modelos são gerados a partir das informações e técnicas obtidas com o desenvolvimento até então. Se comparado com as técnicas apresentadas no arquivo Trial_And_Error, aqui as features usadas independem do foco ser achar o máximo ou o mínimo. Além disso, a transformação em gaussiana foi preferível à transformação em min-max. O modelo usado ainda é o Lasso apesar das limitações.

Cada arquivo é salvo numa pasta chamada persistence com o nome da empresa seguido de "high" ou "min" dependendo do que o modelo foi treinado para prever.

In [12]:
porcentagem_teste = 20
timesplit= TimeSeriesSplit(n_splits=int(100/porcentagem_teste)-1)
features = ["Date","Open", "High","Close","Low", "Date-1"]
scaler = preprocessing.StandardScaler()
clf = linear_model.Lasso(max_iter = 10000)

for name in name_list:
    example_1 = test_frame[name]
    feature_transform = scaler.fit_transform(example_1[features])
    feature_transform = pd.DataFrame(columns=features, data=feature_transform, index=example_1.index)
    
    output_var = pd.DataFrame(example_1["High-1"])
    for train_index, test_index in timesplit.split(feature_transform):
            X_train, X_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
            y_train, y_test = output_var[:len(train_index)].values.ravel(), output_var[len(train_index): (len(train_index)+len(test_index))].values.ravel()
    clf.fit(X_train, y_train)
    dump(clf, f'./persistence/{name}_high.joblib')
    
    output_var = pd.DataFrame(example_1[ "Low-1"])
    for train_index, test_index in timesplit.split(feature_transform):
            X_train, X_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
            y_train, y_test = output_var[:len(train_index)].values.ravel(), output_var[len(train_index): (len(train_index)+len(test_index))].values.ravel()

    clf.fit(X_train, y_train)
    dump(clf, f'./persistence/{name}_low.joblib')